In [2]:
from backbone_cnn import BackboneNet
import torch.functional as F
import torch
import matplotlib.pyplot as plt
import warnings

In [3]:
import pandas as pd
import numpy as np
net = BackboneNet()
X = pd.read_csv('../data/train.csv')
labels = np.array(X['label'])
labels = torch.tensor(labels, dtype=torch.long)
X = np.array(X.drop(columns=['label']))/255
X = X.reshape(-1,1,28,28)
X = torch.tensor(X, dtype=torch.float32)

In [4]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

In [5]:
criterion = nn.CrossEntropyLoss()
labels

tensor([1, 0, 1,  ..., 7, 6, 9])

In [6]:
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [10]:
running_loss = 0.0
for epoch in range(2):
    
    for i in tqdm(range(0, len(labels), 100)):
        
        
        batch_x, batch_labels = X[i:i+100], labels[i:i+100]
        
        output = net(batch_x)
    
        loss = criterion(output, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    running_loss += loss.item()



100%|██████████| 420/420 [00:54<00:00,  7.99it/s]


In [11]:

loss.item()

0.001615996239706874

In [12]:
accuracy = np.mean([i.argmax()==j for i,j in zip(output, batch_labels)])

In [13]:
output[0].argmax()

tensor(4)

In [14]:
accuracy

1.0

In [15]:
len(labels)

42000

In [17]:
batch_labels[0]

tensor(4)

In [18]:
accuracy

1.0